In [1]:
import sys
sys.path.append('../RecSysRep/')

In [2]:
import Basics.Load as ld
import scipy.sparse as sps

URM_all, ICM1, ICM2, ICM3, ICM4 = ld.getCOOs()
ICML = ld.getICMlength('5km')
ICM_weighted = ld.getICMselected('7')
ICM_gen = sps.hstack([ICM1, ICM2])
ICM_all = sps.hstack([ICM1, ICM2, ICM3, ICML])
# URM_train, URM_val = ld.getSplit(URM_train_val, 5678, 0.8)

In [3]:
from Evaluation.Evaluator import EvaluatorHoldout
from Data_manager.split_functions.split_train_validation_random_holdout import split_train_in_two_percentage_global_sample

URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_all, train_percentage = 0.7, seed=6180)
# URM_train, URM_validation = split_train_in_two_percentage_global_sample(URM_train_validation, train_percentage = 0.8)

evaluator_validation = EvaluatorHoldout(URM_validation, cutoff_list=[10])
# evaluator_test = EvaluatorHoldout(URM_test, cutoff_list=[10])

EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions


In [4]:
import os

ofp = "../models_temp/Similarity_Hybrid/seed6180/"

models_to_combine_best = {
                            'EASE': {"topK": None, "normalize_matrix": False, "l2_norm": 3749.2876352484},
                            'IALS_Hyb' : {"num_factors": 28, "epochs": 10, "confidence_scaling": "linear", "alpha": 0.43657990940994623, "epsilon": 0.35472063248578317, "reg": 0.0001698292271931609, "mw": 0.06122362507952762},
                            'RP3ICMnew': {'alpha': 1.029719677583138, 'beta': 1.0630164752134375, 'topK': 6964, 'normalize_similarity': True},
                            'RP3ICM' : {"topK": 2550, "alpha": 1.3058102610510849, "beta": 0.5150718337969987, "normalize_similarity": True, "implicit": True},
                            'IALS' :  {'num_factors': 34, 'epochs': 599, 'confidence_scaling': 'linear', 'alpha': 0.003519435539271083, 'epsilon': 0.09222402080721787, 'reg': 2.4127708108457617e-05},
                            'SLIMgensub': {"l1_ratio" : 0.025887359156206147, "topK": 2140, "alpha": 0.009567288586539689, "workers": 8, "mw": 1},
                            'SLIMBPR' : {"epochs": 440, "lambda_i": 0.007773815998802306, "lambda_j": 0.003342522366982381, "learning_rate": 0.010055161410725193, "topK": 4289, "random_seed": 1234, "sgd_mode": "sgd"},
                            'SLIMweig': {'l1_ratio': 0.0005247075138160404, 'topK': 4983, 'alpha': 0.06067400905430761, 'workers': 8, 'mw': 2.308619939318322},
                            'SLIMER': {'topK': 6000, 'l1_ratio': 0.0005495104968035837, 'alpha': 0.08007142704041009, 'workers': 8},
                            'P3alpha': {'topK': 4834, 'alpha': 1.764994849187595, 'normalize_similarity': True, 'implicit': True},
                            'RP3beta': {"topK": 1049, "alpha": 1.1626473723475605, "beta": 0.6765017195261293, "normalize_similarity": True, "implicit": True},
                            'icm_all' : {"shrink": 5675, "topK": 2310, "feature_weighting": "BM25", "normalize": False},
                            'icm_weighted': {"shrink": 4000, "topK": 985, "feature_weighting": "TF-IDF", "normalize": True},
                            'SLIM_all' : {'l1_ratio': 0.0001043005302985496, 'topK': 1135, 'alpha': 0.06322495726710943, 'workers': 8, 'mw': 1.2937257828415842},
                            'SLIM_all2' : {'topK': 4997, 'l1_ratio': 0.00036065147205858187, 'alpha': 0.03203200564199538, 'mw': 1.0785132020354402, 'workers': 8},
                            'UserKNN' : {"topK": 469, "similarity": "cosine", "shrink": 588, "normalize": True, "feature_weighting": "TF-IDF", "URM_bias": False},
                            'SLIM_BPR_Hyb' : {"epochs": 1443, "lambda_i": 8.900837513818856e-05, "lambda_j": 1.2615223007492727e-05, "learning_rate": 0.0037706733838839264, "topK": 6181, "random_seed": 1234, "sgd_mode": "sgd"},
                            'Neural': {"epochs": 295, "learning_rate": 9.598708662230564e-05, "l2_reg": 1.4122381520119457e-05, "dropout": 0.5759445339123964, "total_anneal_steps": 147387, "anneal_cap": 0.4376000433422989, "batch_size": 256, "encoding_size": 185, "next_layer_size_multiplier": 6, "max_n_hidden_layers": 1, "max_layer_size": 5000.0}
                         }


In [5]:
def model_init(recommender, name, args):
    path_name = ofp + name + ".zip"
    print(path_name)
    if os.path.exists(path_name):
        print("Model found!")
        recommender.load_model(ofp, name)
    else:
        print("Model does not exists, creating...")
        if not os.path.exists(ofp):
            print("Main folder does not exist, creating...")
            os.makedirs(ofp)
        recommender.fit(**args)
        recommender.save_model(ofp, name)

In [6]:
from Recommenders.NonPersonalizedRecommender import TopPop
from Recommenders.KNN.ItemKNNCBFRecommender import ItemKNNCBFRecommender
from Recommenders.EASE_R.EASE_R_Recommender import EASE_R_Recommender
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython
from Recommenders.SLIM.Cython.SLIM_BPR_Cython import SLIM_BPR_Cython_Hybrid
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import SLIMElasticNetRecommender
from Recommenders.SLIM.SLIMElasticNetRecommender import MultiThreadSLIM_SLIM_S_ElasticNetRecommender
from Recommenders.GraphBased.P3alphaRecommender import P3alphaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommender
from Recommenders.GraphBased.RP3betaRecommender import RP3betaRecommenderICM
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender
from Recommenders.MatrixFactorization.IALSRecommender import IALSRecommender_Hybrid
from Recommenders.KNN.UserKNNCFRecommender import UserKNNCFRecommender
from Recommenders.Neural.MultVAERecommender import MultVAERecommender
from Recommenders.MatrixFactorization.PureSVDRecommender import PureSVDRecommender

In [7]:
from Recommenders.KNN.ItemKNNSimilarityHybridRecommender import ItemKNNSimilarityHybridRecommender
from Recommenders.KNN.ItemKNNSimilarityHybridRecommenderNormal import ItemKNNSimilarityHybridRecommenderNormal
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridMultipleRecommender
from Recommenders.KNN.ItemKNNScoresHybridMultipleRecommender import ItemKNNScoresHybridTwoRecommender
from Recommenders.HybridScores.DifferentStructure import TwoDifferentModelRecommender

In [8]:
groups = ld.load_groups(5)

In [9]:
recommender_slim_all = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_all2 = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_all)
recommender_slim_wei = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_train, ICM_weighted)
recommender_ease = EASE_R_Recommender(URM_train)
recommender_itemknn = ItemKNNCBFRecommender(URM_train, ICM_all)
recommender_rp3 = RP3betaRecommender(URM_train)
#recommender_svd = PureSVDRecommender(URM_train)

model_init(recommender_slim_all, 'SLIMall6180', models_to_combine_best['SLIM_all'])
model_init(recommender_slim_all2, 'SLIMall26180', models_to_combine_best['SLIM_all2'])
model_init(recommender_slim_wei, 'SLIMweig6180', models_to_combine_best['SLIMweig'])
model_init(recommender_ease, 'EASE6180', models_to_combine_best['EASE'])
model_init(recommender_rp3, 'RP3beta6180', models_to_combine_best['RP3beta'])
model_init(recommender_itemknn, 'ICM_all6180', models_to_combine_best['icm_all'])
#model_init(recommender_svd, 'PureSVD6180', {'num_factors': 30})

recommender_slim_all.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_all2.RECOMMENDER_NAME = 'SLIMall2'
recommender_slim_wei.RECOMMENDER_NAME = 'SLIMweig'
recommender_ease.RECOMMENDER_NAME = 'EASE'
recommender_itemknn.RECOMMENDER_NAME = 'KNN'
recommender_rp3.RECOMMENDER_NAME = 'RP3'
#recommender_svd.RECOMMENDER_NAME = 'SVD'

../models_temp/Similarity_Hybrid/seed6180/SLIMall6180.zip
Model does not exists, creating...


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [16:53<00:00, 17.82it/s]


SLIMElasticNetRecommender: Saving model in file '../models_temp/Similarity_Hybrid/seed6180/SLIMall6180'
SLIMElasticNetRecommender: Saving complete
../models_temp/Similarity_Hybrid/seed6180/SLIMall26180.zip
Model does not exists, creating...


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [14:36<00:00, 20.60it/s]


SLIMElasticNetRecommender: Saving model in file '../models_temp/Similarity_Hybrid/seed6180/SLIMall26180'
SLIMElasticNetRecommender: Saving complete
../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180.zip
Model does not exists, creating...


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [14:04<00:00, 21.37it/s]


SLIMElasticNetRecommender: Saving model in file '../models_temp/Similarity_Hybrid/seed6180/SLIMweig6180'
SLIMElasticNetRecommender: Saving complete
../models_temp/Similarity_Hybrid/seed6180/EASE6180.zip
Model does not exists, creating...
EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 1549.24 column/sec. Elapsed time 11.66 sec
EASE_R_Recommender: Fitting model... done in 1.47 min
EASE_R_Recommender: Saving model in file '../models_temp/Similarity_Hybrid/seed6180/EASE6180'
EASE_R_Recommender: Saving complete
../models_temp/Similarity_Hybrid/seed6180/RP3beta6180.zip
Model found!
RP3betaRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/RP3beta6180'
RP3betaRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/ICM_all6180.zip
Model found!
ItemKNNCBFRecommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/ICM_all6180'
ItemKNNCBFRecommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180

In [11]:
recommender_slim_bpr = SLIM_BPR_Cython(URM_train)
recommender_slim_bpr_hyb = SLIM_BPR_Cython_Hybrid(URM_train, ICM_weighted)
recommender_ials_hyb = IALSRecommender_Hybrid(URM_train, ICM_weighted)
recommender_p3alpha = P3alphaRecommender(URM_train)
recommender_slimer = MultiThreadSLIM_SLIMElasticNetRecommender(URM_train)

model_init(recommender_slim_bpr, 'SLIMBPR6180', models_to_combine_best['SLIMBPR'])
model_init(recommender_slim_bpr_hyb, 'SLIMBPR_Hyb6180', models_to_combine_best['SLIM_BPR_Hyb'])
model_init(recommender_ials_hyb, 'IALS_Hyb6180', models_to_combine_best['IALS_Hyb'])
model_init(recommender_p3alpha, 'P3alpha6180', models_to_combine_best['P3alpha'])
model_init(recommender_slimer, 'SLIMER6180', models_to_combine_best['SLIMER'])

recommender_slim_bpr.RECOMMENDER_NAME = 'SLIMall'
recommender_slim_bpr_hyb.RECOMMENDER_NAME = 'SLIMall2'
recommender_ials_hyb.RECOMMENDER_NAME = 'IALS_Hyb'
recommender_p3alpha.RECOMMENDER_NAME = 'P3alpha'
recommender_slimer.RECOMMENDER_NAME = 'SLIMER'

../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180.zip
Model found!
SLIM_BPR_Recommender: Loading model from file '../models_temp/Similarity_Hybrid/seed6180/SLIMBPR6180'
SLIM_BPR_Recommender: Loading complete
../models_temp/Similarity_Hybrid/seed6180/SLIMBPR_Hyb6180.zip
Model does not exists, creating...
Unable to read memory status: list index out of range
SLIM_BPR_Recommender_Hybrid: Automatic selection of fastest train mode. Unable to get current RAM status, you may be using a non-Linux operating system. Using dense matrix.
Processed 13887 (100.0%) in 1.03 sec. BPR loss is 1.83E-01. Sample per second: 13488
SLIM_BPR_Recommender_Hybrid: Epoch 1 of 1443. Elapsed time 0.40 sec
Processed 13887 (100.0%) in 0.42 sec. BPR loss is 8.08E-01. Sample per second: 33044
SLIM_BPR_Recommender_Hybrid: Epoch 2 of 1443. Elapsed time 0.79 sec
Processed 13887 (100.0%) in 0.83 sec. BPR loss is 1.48E+00. Sample per second: 16753
SLIM_BPR_Recommender_Hybrid: Epoch 3 of 1443. Elapsed time 1.19 sec
Proce

Processed 13887 (100.0%) in 1.04 sec. BPR loss is 1.35E+01. Sample per second: 13382
SLIM_BPR_Recommender_Hybrid: Epoch 51 of 1443. Elapsed time 20.40 sec
Processed 13887 (100.0%) in 0.43 sec. BPR loss is 1.35E+01. Sample per second: 32074
SLIM_BPR_Recommender_Hybrid: Epoch 52 of 1443. Elapsed time 20.80 sec
Processed 13887 (100.0%) in 0.83 sec. BPR loss is 1.39E+01. Sample per second: 16648
SLIM_BPR_Recommender_Hybrid: Epoch 53 of 1443. Elapsed time 21.20 sec
Processed 13887 (100.0%) in 1.27 sec. BPR loss is 1.35E+01. Sample per second: 10953
SLIM_BPR_Recommender_Hybrid: Epoch 54 of 1443. Elapsed time 21.63 sec
Processed 13887 (100.0%) in 0.67 sec. BPR loss is 1.39E+01. Sample per second: 20574
SLIM_BPR_Recommender_Hybrid: Epoch 55 of 1443. Elapsed time 22.04 sec
Processed 13887 (100.0%) in 1.12 sec. BPR loss is 1.41E+01. Sample per second: 12432
SLIM_BPR_Recommender_Hybrid: Epoch 56 of 1443. Elapsed time 22.48 sec
Processed 13887 (100.0%) in 0.55 sec. BPR loss is 1.43E+01. Sample per

Processed 13887 (100.0%) in 1.10 sec. BPR loss is 1.74E+01. Sample per second: 12626
SLIM_BPR_Recommender_Hybrid: Epoch 104 of 1443. Elapsed time 41.47 sec
Processed 13887 (100.0%) in 0.51 sec. BPR loss is 1.72E+01. Sample per second: 27399
SLIM_BPR_Recommender_Hybrid: Epoch 105 of 1443. Elapsed time 41.87 sec
Processed 13887 (100.0%) in 0.91 sec. BPR loss is 1.72E+01. Sample per second: 15281
SLIM_BPR_Recommender_Hybrid: Epoch 106 of 1443. Elapsed time 42.27 sec
Processed 13887 (100.0%) in 1.31 sec. BPR loss is 1.75E+01. Sample per second: 10599
SLIM_BPR_Recommender_Hybrid: Epoch 107 of 1443. Elapsed time 42.67 sec
Processed 13887 (100.0%) in 0.71 sec. BPR loss is 1.77E+01. Sample per second: 19616
SLIM_BPR_Recommender_Hybrid: Epoch 108 of 1443. Elapsed time 43.07 sec
Processed 13887 (100.0%) in 1.12 sec. BPR loss is 1.81E+01. Sample per second: 12390
SLIM_BPR_Recommender_Hybrid: Epoch 109 of 1443. Elapsed time 43.49 sec
Processed 13887 (100.0%) in 0.52 sec. BPR loss is 1.75E+01. Samp

Processed 13887 (100.0%) in 1.20 sec. BPR loss is 1.93E+01. Sample per second: 11563
SLIM_BPR_Recommender_Hybrid: Epoch 157 of 1443. Elapsed time 1.04 min
Processed 13887 (100.0%) in 0.60 sec. BPR loss is 1.92E+01. Sample per second: 23300
SLIM_BPR_Recommender_Hybrid: Epoch 158 of 1443. Elapsed time 1.05 min
Processed 13887 (100.0%) in 1.00 sec. BPR loss is 1.94E+01. Sample per second: 13871
SLIM_BPR_Recommender_Hybrid: Epoch 159 of 1443. Elapsed time 1.06 min
Processed 13887 (100.0%) in 0.40 sec. BPR loss is 1.91E+01. Sample per second: 34609
SLIM_BPR_Recommender_Hybrid: Epoch 160 of 1443. Elapsed time 1.06 min
Processed 13887 (100.0%) in 0.79 sec. BPR loss is 1.96E+01. Sample per second: 17455
SLIM_BPR_Recommender_Hybrid: Epoch 161 of 1443. Elapsed time 1.07 min
Processed 13887 (100.0%) in 1.24 sec. BPR loss is 1.98E+01. Sample per second: 11238
SLIM_BPR_Recommender_Hybrid: Epoch 162 of 1443. Elapsed time 1.08 min
Processed 13887 (100.0%) in 0.64 sec. BPR loss is 2.05E+01. Sample per

Processed 13887 (100.0%) in 1.04 sec. BPR loss is 2.06E+01. Sample per second: 13384
SLIM_BPR_Recommender_Hybrid: Epoch 210 of 1443. Elapsed time 1.39 min
Processed 13887 (100.0%) in 0.43 sec. BPR loss is 2.07E+01. Sample per second: 32548
SLIM_BPR_Recommender_Hybrid: Epoch 211 of 1443. Elapsed time 1.40 min
Processed 13887 (100.0%) in 0.81 sec. BPR loss is 2.05E+01. Sample per second: 17137
SLIM_BPR_Recommender_Hybrid: Epoch 212 of 1443. Elapsed time 1.40 min
Processed 13887 (100.0%) in 1.20 sec. BPR loss is 2.12E+01. Sample per second: 11530
SLIM_BPR_Recommender_Hybrid: Epoch 213 of 1443. Elapsed time 1.41 min
Processed 13887 (100.0%) in 0.60 sec. BPR loss is 2.03E+01. Sample per second: 23157
SLIM_BPR_Recommender_Hybrid: Epoch 214 of 1443. Elapsed time 1.42 min
Processed 13887 (100.0%) in 0.98 sec. BPR loss is 2.06E+01. Sample per second: 14129
SLIM_BPR_Recommender_Hybrid: Epoch 215 of 1443. Elapsed time 1.42 min
Processed 13887 (100.0%) in 1.37 sec. BPR loss is 2.05E+01. Sample per

...

SLIM_BPR_Recommender_Hybrid: Epoch 1216 of 1443. Elapsed time 8.16 min
Processed 13887 (100.0%) in 0.47 sec. BPR loss is 2.77E+01. Sample per second: 29290
SLIM_BPR_Recommender_Hybrid: Epoch 1217 of 1443. Elapsed time 8.16 min
Processed 13887 (100.0%) in 0.91 sec. BPR loss is 2.73E+01. Sample per second: 15341
SLIM_BPR_Recommender_Hybrid: Epoch 1218 of 1443. Elapsed time 8.17 min
Processed 13887 (100.0%) in 1.34 sec. BPR loss is 2.77E+01. Sample per second: 10392
SLIM_BPR_Recommender_Hybrid: Epoch 1219 of 1443. Elapsed time 8.18 min
Processed 13887 (100.0%) in 0.76 sec. BPR loss is 2.67E+01. Sample per second: 18152
SLIM_BPR_Recommender_Hybrid: Epoch 1220 of 1443. Elapsed time 8.19 min
Processed 13887 (100.0%) in 1.20 sec. BPR loss is 2.73E+01. Sample per second: 11610
SLIM_BPR_Recommender_Hybrid: Epoch 1221 of 1443. Elapsed time 8.19 min
Processed 13887 (100.0%) in 0.61 sec. BPR loss is 2.68E+01. Sample per second: 22675
SLIM_BPR_Recommender_Hybrid: Epoch 1222 of 1443. Elapsed time 8.

SLIM_BPR_Recommender_Hybrid: Epoch 1269 of 1443. Elapsed time 8.53 min
Processed 13887 (100.0%) in 1.03 sec. BPR loss is 2.79E+01. Sample per second: 13477
SLIM_BPR_Recommender_Hybrid: Epoch 1270 of 1443. Elapsed time 8.54 min
Processed 13887 (100.0%) in 0.46 sec. BPR loss is 2.75E+01. Sample per second: 30157
SLIM_BPR_Recommender_Hybrid: Epoch 1271 of 1443. Elapsed time 8.55 min
Processed 13887 (100.0%) in 0.89 sec. BPR loss is 2.71E+01. Sample per second: 15520
SLIM_BPR_Recommender_Hybrid: Epoch 1272 of 1443. Elapsed time 8.55 min
Processed 13887 (100.0%) in 1.33 sec. BPR loss is 2.72E+01. Sample per second: 10474
SLIM_BPR_Recommender_Hybrid: Epoch 1273 of 1443. Elapsed time 8.56 min
Processed 13887 (100.0%) in 0.76 sec. BPR loss is 2.74E+01. Sample per second: 18273
SLIM_BPR_Recommender_Hybrid: Epoch 1274 of 1443. Elapsed time 8.57 min
Processed 13887 (100.0%) in 1.19 sec. BPR loss is 2.74E+01. Sample per second: 11697
SLIM_BPR_Recommender_Hybrid: Epoch 1275 of 1443. Elapsed time 8.

SLIM_BPR_Recommender_Hybrid: Epoch 1322 of 1443. Elapsed time 8.91 min
Processed 13887 (100.0%) in 0.68 sec. BPR loss is 2.74E+01. Sample per second: 20521
SLIM_BPR_Recommender_Hybrid: Epoch 1323 of 1443. Elapsed time 8.92 min
Processed 13887 (100.0%) in 1.08 sec. BPR loss is 2.78E+01. Sample per second: 12813
SLIM_BPR_Recommender_Hybrid: Epoch 1324 of 1443. Elapsed time 8.92 min
Processed 13887 (100.0%) in 0.51 sec. BPR loss is 2.75E+01. Sample per second: 26969
SLIM_BPR_Recommender_Hybrid: Epoch 1325 of 1443. Elapsed time 8.93 min
Processed 13887 (100.0%) in 0.93 sec. BPR loss is 2.72E+01. Sample per second: 14842
SLIM_BPR_Recommender_Hybrid: Epoch 1326 of 1443. Elapsed time 8.94 min
Processed 13887 (100.0%) in 1.35 sec. BPR loss is 2.68E+01. Sample per second: 10273
SLIM_BPR_Recommender_Hybrid: Epoch 1327 of 1443. Elapsed time 8.95 min
Processed 13887 (100.0%) in 0.78 sec. BPR loss is 2.76E+01. Sample per second: 17799
SLIM_BPR_Recommender_Hybrid: Epoch 1328 of 1443. Elapsed time 8.

SLIM_BPR_Recommender_Hybrid: Epoch 1375 of 1443. Elapsed time 9.29 min
Processed 13887 (100.0%) in 1.19 sec. BPR loss is 2.74E+01. Sample per second: 11639
SLIM_BPR_Recommender_Hybrid: Epoch 1376 of 1443. Elapsed time 9.29 min
Processed 13887 (100.0%) in 0.62 sec. BPR loss is 2.76E+01. Sample per second: 22389
SLIM_BPR_Recommender_Hybrid: Epoch 1377 of 1443. Elapsed time 9.30 min
Processed 13887 (100.0%) in 1.05 sec. BPR loss is 2.74E+01. Sample per second: 13213
SLIM_BPR_Recommender_Hybrid: Epoch 1378 of 1443. Elapsed time 9.31 min
Processed 13887 (100.0%) in 0.48 sec. BPR loss is 2.79E+01. Sample per second: 29171
SLIM_BPR_Recommender_Hybrid: Epoch 1379 of 1443. Elapsed time 9.31 min
Processed 13887 (100.0%) in 0.90 sec. BPR loss is 2.75E+01. Sample per second: 15404
SLIM_BPR_Recommender_Hybrid: Epoch 1380 of 1443. Elapsed time 9.32 min
Processed 13887 (100.0%) in 1.33 sec. BPR loss is 2.81E+01. Sample per second: 10413
SLIM_BPR_Recommender_Hybrid: Epoch 1381 of 1443. Elapsed time 9.

SLIM_BPR_Recommender_Hybrid: Epoch 1428 of 1443. Elapsed time 9.66 min
Processed 13887 (100.0%) in 0.78 sec. BPR loss is 2.80E+01. Sample per second: 17771
SLIM_BPR_Recommender_Hybrid: Epoch 1429 of 1443. Elapsed time 9.67 min
Processed 13887 (100.0%) in 1.21 sec. BPR loss is 2.81E+01. Sample per second: 11466
SLIM_BPR_Recommender_Hybrid: Epoch 1430 of 1443. Elapsed time 9.68 min
Processed 13887 (100.0%) in 0.64 sec. BPR loss is 2.82E+01. Sample per second: 21758
SLIM_BPR_Recommender_Hybrid: Epoch 1431 of 1443. Elapsed time 9.68 min
Processed 13887 (100.0%) in 1.07 sec. BPR loss is 2.84E+01. Sample per second: 12953
SLIM_BPR_Recommender_Hybrid: Epoch 1432 of 1443. Elapsed time 9.69 min
Processed 13887 (100.0%) in 0.50 sec. BPR loss is 2.80E+01. Sample per second: 27595
SLIM_BPR_Recommender_Hybrid: Epoch 1433 of 1443. Elapsed time 9.70 min
Processed 13887 (100.0%) in 0.92 sec. BPR loss is 2.85E+01. Sample per second: 15070
SLIM_BPR_Recommender_Hybrid: Epoch 1434 of 1443. Elapsed time 9.

In [18]:
def rank_models(evaluator, recommenders):
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        print(r.RECOMMENDER_NAME, r_d.loc[10]['MAP'])
        
def rank_models_group_wise(evaluators, recommenders):
    i = 0 # group
    dict_res = {}
    for e in evaluators:
        dict_res[i] = {}
        for r in recommenders:
            r_d, _ = e.evaluateRecommender(r)
            dict_res[i][r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        i+=1
    return dict_res

In [19]:
def rank_models(evaluator, recommenders):
    dict_res = {}
    for r in recommenders:
        r_d, _ = evaluator.evaluateRecommender(r)
        dict_res[r.RECOMMENDER_NAME] = r_d.loc[10]['MAP']
        
    return dict_res

In [28]:
import numpy as np

evaluators = []
n_groups = 5
cutoff = 10
n_users = URM_all.shape[0]

for group_id in range(0, n_groups):
    users_in_group = groups[group_id]
    print("GROUP:", group_id, "NUMBER:", len(users_in_group))
    all_users = np.arange(0, n_users)
    users_not_in_group = np.delete(all_users, users_in_group)

    evaluator_test = EvaluatorHoldout(URM_validation, cutoff_list=[cutoff], ignore_users=users_not_in_group)
    evaluators.append(evaluator_test)

GROUP: 0 NUMBER: 3066
EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 10584 Users
GROUP: 1 NUMBER: 389
EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 13261 Users
GROUP: 2 NUMBER: 4121
EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9529 Users
GROUP: 3 NUMBER: 1566
EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 12084 Users
GROUP: 4 NUMBER: 4508
EvaluatorHoldout: Ignoring 13649 ( 0.0%) Users that have less than 1 test interactions
EvaluatorHoldout: Ignoring 9142 Users


In [29]:
recommender_slim_all2_ease = TwoDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease)
recommender_slim_all2_ease.fit(None, 0.52)

In [30]:
all_rec = [recommender_slim_all, recommender_slim_all2, recommender_slim_wei, recommender_ease, recommender_itemknn, recommender_rp3, recommender_svd, recommender_slim_bpr, recommender_slim_bpr_hyb, recommender_ials_hyb, recommender_slimer, recommender_slim_all2_ease]
dict_res = rank_models(evaluator_validation, all_rec)
dict_res2 = rank_models_group_wise(evaluators, all_rec)

EvaluatorHoldout: Processed 13649 (100.0%) in 22.82 sec. Users per second: 598
             MAP
cutoff          
10      0.342705
EvaluatorHoldout: Processed 13649 (100.0%) in 20.80 sec. Users per second: 656
             MAP
cutoff          
10      0.342397
EvaluatorHoldout: Processed 13649 (100.0%) in 22.82 sec. Users per second: 598
             MAP
cutoff          
10      0.342891
EvaluatorHoldout: Processed 13649 (100.0%) in 41.39 sec. Users per second: 330
             MAP
cutoff          
10      0.338931
EvaluatorHoldout: Processed 13649 (100.0%) in 36.20 sec. Users per second: 377
             MAP
cutoff          
10      0.129848
EvaluatorHoldout: Processed 13649 (100.0%) in 28.12 sec. Users per second: 485
             MAP
cutoff          
10      0.307918
EvaluatorHoldout: Processed 13649 (100.0%) in 5.78 sec. Users per second: 2363
            MAP
cutoff         
10      0.32064
EvaluatorHoldout: Processed 13649 (100.0%) in 46.89 sec. Users per second: 291
             M

EvaluatorHoldout: Processed 4508 (100.0%) in 12.57 sec. Users per second: 359
             MAP
cutoff          
10      0.137651
EvaluatorHoldout: Processed 4508 (100.0%) in 9.73 sec. Users per second: 464
             MAP
cutoff          
10      0.332699
EvaluatorHoldout: Processed 4508 (100.0%) in 1.43 sec. Users per second: 3152
             MAP
cutoff          
10      0.346008
EvaluatorHoldout: Processed 4508 (100.0%) in 15.13 sec. Users per second: 298
             MAP
cutoff          
10      0.339687
EvaluatorHoldout: Processed 4508 (100.0%) in 22.43 sec. Users per second: 201
             MAP
cutoff          
10      0.347491
EvaluatorHoldout: Processed 4508 (100.0%) in 2.59 sec. Users per second: 1742
             MAP
cutoff          
10      0.346384
EvaluatorHoldout: Processed 4508 (100.0%) in 8.59 sec. Users per second: 525
            MAP
cutoff         
10      0.36633
EvaluatorHoldout: Processed 4508 (100.0%) in 17.12 sec. Users per second: 263
             MAP
cutoff 

In [31]:
for i in dict_res2.keys():
    dict_res2[i] = {k: v for k, v in sorted(dict_res2[i].items(), key=lambda item: item[1])}
    print(i, dict_res2[i])
    
dict_res = {k: v for k, v in sorted(dict_res.items(), key=lambda item: item[1])}
print(dict_res)

0 {'KNN': 0.154956292258151, 'SLIMall': 0.36631439029188534, 'RP3': 0.3676907376344753, 'SVD': 0.36778595760983135, 'IALS_Hyb': 0.37048981921535756, 'SLIMall2': 0.3707417632197469, 'EASE': 0.3891332096004312, 'SLIMweig': 0.39029160065852875, 'SLIMER': 0.39110664585468835, 'DifferentLossScoresHybridRecommender': 0.39143966338437985}
1 {'KNN': 0.18479301832129605, 'SLIMall2': 0.3704631329824134, 'SLIMall': 0.3744785163422692, 'RP3': 0.39152650263190086, 'SVD': 0.39240482311176406, 'IALS_Hyb': 0.39274390990329294, 'DifferentLossScoresHybridRecommender': 0.39859162688211475, 'SLIMER': 0.4009925735504139, 'SLIMweig': 0.4020430897294653, 'EASE': 0.4035332762067981}
2 {'KNN': 0.08167367853290203, 'RP3': 0.2028048620742791, 'SVD': 0.2281147807828632, 'IALS_Hyb': 0.22990222877176678, 'SLIMall': 0.23919592772384018, 'SLIMall2': 0.24800922715364426, 'EASE': 0.24834709701032495, 'SLIMweig': 0.25492782978887385, 'SLIMER': 0.2551878178455849, 'DifferentLossScoresHybridRecommender': 0.256367766605024

In [32]:
from Recommenders.HybridScores.DifferentStructure import ThreeDifferentModelRecommender
import optuna

class Objective(object):
    def __init__(self, URM_train, recommender1, recommender2, recommender3, evaluator):
        # Hold this implementation specific arguments as the fields of the class.
        self.URM_train = URM_train
        self.evaluator = evaluator
        self.recommender_1 = recommender1
        self.recommender_2 = recommender2
        self.recommender_3 = recommender3

    def __call__(self, trial):
        # Calculate an objective value by using the extra arguments.
        w_1 = trial.suggest_uniform('w_1', 0, 1)
        w_2 = trial.suggest_uniform('w_2', 0, 1)
        w_3 = trial.suggest_uniform('w_3', 0, 1)

        recommender_final = ThreeDifferentModelRecommender(self.URM_train.tocsr(), self.recommender_1, self.recommender_2, self.recommender_3)
        recommender_final.fit(None, w_1, w_2, w_3)
        result_dict, _ = self.evaluator.evaluateRecommender(recommender_final)

        map_v = -result_dict.loc[10]['MAP']
        if map_v >= -0.245: # minimum acceptable map
            # Calculate the penalty.
            trial.report(map_v, 0)

            if trial.should_prune():
                raise optuna.exceptions.TrialPruned()

        return map_v

In [ ]:
study = optuna.create_study(direction='minimize')
study.optimize(Objective(URM_train, recommender_slimer, recommender_ease, recommender_rp3, evaluator_validation), n_trials=500)

print(study.best_params)

[I 2022-01-07 09:38:23,140] A new study created in memory with name: no-name-42c7b60d-6586-441b-952e-7fcc2002d463


CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.5647792446347851
EASE with weight beta: 0.07970390891734831
RP3 with weight gamma: 0.35551684644786663
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 1.58 min. Users per second: 144


[I 2022-01-07 09:39:57,836] Trial 0 finished with value: -0.34398695409978636 and parameters: {'w_1': 0.9134638499069727, 'w_2': 0.12891167687891247, 'w_3': 0.5750065894738311}. Best is trial 0 with value: -0.34398695409978636.


             MAP
cutoff          
10      0.343987
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.40993646094362396
EASE with weight beta: 0.04079255359430407
RP3 with weight gamma: 0.549270985462072
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 1.52 min. Users per second: 149


[I 2022-01-07 09:41:29,403] Trial 1 finished with value: -0.34376284034995347 and parameters: {'w_1': 0.3193600185990084, 'w_2': 0.03177934122910231, 'w_3': 0.4279082463884245}. Best is trial 0 with value: -0.34398695409978636.


...

             MAP
cutoff          
10      0.343941
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.46104177770458854
EASE with weight beta: 0.36154008810756627
RP3 with weight gamma: 0.17741813418784522
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 2.00 min. Users per second: 114
             MAP
cutoff          
10      0.343889


[I 2022-01-07 14:08:35,805] Trial 167 finished with value: -0.34388865630019017 and parameters: {'w_1': 0.9547999523801585, 'w_2': 0.7487357450061883, 'w_3': 0.36742619490434353}. Best is trial 162 with value: -0.34463348381822395.


CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.3840716066071872
EASE with weight beta: 0.3361834987641381
RP3 with weight gamma: 0.2797448946286747
Norm type: None
EvaluatorHoldout: Processed 13649 (100.0%) in 4.15 min. Users per second: 55


[I 2022-01-07 14:12:45,190] Trial 168 finished with value: -0.3440712297080921 and parameters: {'w_1': 0.4534755936497039, 'w_2': 0.39693382446055875, 'w_3': 0.3302964342583043}. Best is trial 162 with value: -0.34463348381822395.


             MAP
cutoff          
10      0.344071
CURRENT CONFIGURATION:
SLIMER with weight alpha: 0.5226217441061822
EASE with weight beta: 0.2246665626709774
RP3 with weight gamma: 0.2527116932228403
Norm type: None
EvaluatorHoldout: Processed 8000 (58.6%) in 18.69 min. Users per second: 7
EvaluatorHoldout: Processed 9000 (65.9%) in 31.64 min. Users per second: 5
EvaluatorHoldout: Processed 10000 (73.3%) in 39.00 min. Users per second: 4


In [22]:
best_hyp = {'norm': 2, 'alpha': 0.4583808522548663, 'beta': 0.09643193702948973, 'gamma': 0.10698567245567658}
recommender_rp3_slim_all2_ease = ThreeDifferentModelRecommender(URM_train, recommender_slim_all2, recommender_ease, recommender_rp3)
recommender_rp3_slim_all2_ease.fit(**best_hyp)
recommender_rp3_slim_all2_ease.RECOMMENDER_NAME = 'SLIMALL2+ease+RP3'

CURRENT CONFIGURATION:
SLIMall2 with weight alpha: 0.692629068749524
EASE with weight beta: 0.14571193891255987
RP3 with weight gamma: 0.16165899233791609
Norm type: 2


In [23]:
import numpy as np

def test_percentage_sim(recommender_a, recommender_b, evaluator, high1, low1, step):
    recommender = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender_a.W_sparse, recommender_b.W_sparse)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

def test_percentage_sco(recommender_a, recommender_b, evaluator, high1, low1, step, norm=None):
    recommender = TwoDifferentModelRecommender(URM_train, recommender_a, recommender_b)
    results = []
    alp_space = np.linspace(high1, low1, step, False)
    for alp in alp_space:
        recommender.fit(norm, alp)
        r_d, _ = evaluator.evaluateRecommender(recommender)
        print(alp, ":", r_d.loc[10]['MAP'])
        results.append(r_d.loc[10]['MAP'])
    
    return alp_space, results

In [ ]:
alp_space, results = test_percentage_sco(recommender_slim_all2, recommender_ease, evaluator_validation, 0.6, 0.76, 10, 2)

In [22]:
best_alp = 0.8997142857142857
recommender_knn_slim = ItemKNNSimilarityHybridRecommenderNormal(URM_train, recommender1.W_sparse, recommender3.W_sparse)
recommender_knn_slim.fit(best_alp)

In [ ]:
alp_space, results = test_percentage_sco(recommender_knn_slim, recommender2, evaluator_validation, 0.79, 0.81, 20) # 0.6305084745762711 : 0.250522906101042

EvaluatorHoldout: Processed 13643 (100.0%) in 42.09 sec. Users per second: 324
             MAP
cutoff          
10      0.250745
0.79 : 0.25074490784855014
EvaluatorHoldout: Processed 13643 (100.0%) in 41.98 sec. Users per second: 325
             MAP
cutoff          
10      0.250766
0.7910526315789475 : 0.2507663066937047
EvaluatorHoldout: Processed 13643 (100.0%) in 41.14 sec. Users per second: 332
             MAP
cutoff          
10      0.250755
0.7921052631578948 : 0.25075466632693705
EvaluatorHoldout: Processed 13643 (100.0%) in 42.41 sec. Users per second: 322
             MAP
cutoff          
10      0.250778
0.7931578947368422 : 0.2507781390305385
EvaluatorHoldout: Processed 13643 (100.0%) in 44.69 sec. Users per second: 305
             MAP
cutoff          
10      0.250799
0.7942105263157895 : 0.2507986216549203
EvaluatorHoldout: Processed 13643 (100.0%) in 47.58 sec. Users per second: 287
             MAP
cutoff          
10      0.250833
0.7952631578947369 : 0.250832754

In [13]:
ofp = "../all_models/URM_all/"

recommender1a = MultiThreadSLIM_SLIM_S_ElasticNetRecommender(URM_all, ICM_all)
recommender2a = EASE_R_Recommender(URM_all)
recommender3a = ItemKNNCBFRecommender(URM_all, ICM_all)

model_init(recommender1a, 'SLIMall', models_to_combine_best['SLIM_all'])
model_init(recommender2a, 'EASE', models_to_combine_best['EASE'])
model_init(recommender3a, 'ItemKNNall', models_to_combine_best['icm_all'])

../all_models/URM_all/SLIMall.zip
Model does not exists, creating...


100%|███████████████████████████████████████████████████████████████████████████▉| 18056/18059 [19:19<00:00, 15.57it/s]


SLIMElasticNetRecommender: Saving model in file '../all_models/URM_all/SLIMall'
SLIMElasticNetRecommender: Saving complete
../all_models/URM_all/EASE.zip
Model does not exists, creating...
EASE_R_Recommender: Fitting model... 
Similarity column 18059 (100.0%), 514.04 column/sec. Elapsed time 35.13 sec
EASE_R_Recommender: Fitting model... done in 4.00 min
EASE_R_Recommender: Saving model in file '../all_models/URM_all/EASE'
EASE_R_Recommender: Saving complete


In [ ]:
best_alp = 0.8997142857142857
recommender_knn_slima = ItemKNNSimilarityHybridRecommenderNormal(URM_all, recommender1a.W_sparse, recommender3a.W_sparse)
recommender_knn_slima.fit(best_alp)

In [ ]:
recommender_final = TwoDifferentModelRecommender(URM_all, recommender_knn_slima, recommender2a)
recommender_final.fit(None, 0.7963157894736842)

In [15]:
import pandas as pd
K=10

user_test_path = '../data/data_target_users_test.csv'
user_test_dataframe = pd.read_csv(filepath_or_buffer=user_test_path,
                                sep=",",
                                dtype={0:int})

subm_set = user_test_dataframe.to_numpy().T[0]


subm_res = {"user_id":[], "item_list":[]}

for user_id in subm_set:
    subm_res["user_id"].append(user_id)
    res = recommender_final.recommend(user_id, K)
    print("USER", user_id, "RECOMMENDED!")
    res = ' '.join(map(str, res))
    print(res)
    subm_res["item_list"].append(res)


# print(subm_res)

submission = pd.DataFrame.from_dict(subm_res)
# submission

from datetime import datetime
now = datetime.now() # current date and time


submission.to_csv('../subs/submission {:%Y_%m_%d %H_%M_%S}.csv'.format(now), index=False)

USER 0 RECOMMENDED!
3899 5433 16365 8465 16359 2243 6061 640 9130 10586
USER 1 RECOMMENDED!
3473 14400 940 16012 2118 9275 16365 640 10118 9373
USER 2 RECOMMENDED!
6177 1551 17022 17330 16365 14229 15119 2748 2846 4607
USER 3 RECOMMENDED!
8194 15551 16359 15082 960 8371 15507 16444 6118 5885
USER 4 RECOMMENDED!
8194 2152 10586 10954 6177 9871 5299 16494 13691 3473
USER 5 RECOMMENDED!
1846 16640 14396 9621 7867 11227 13098 13585 99 5312
USER 6 RECOMMENDED!
6214 985 16846 14358 14181 8207 7606 9621 13443 14868
USER 7 RECOMMENDED!
13963 15119 1885 14031 3433 4292 7751 17364 640 3768
USER 8 RECOMMENDED!
8635 3619 5605 2665 1259 10865 5299 3768 8502 6011
USER 9 RECOMMENDED!
5605 8207 1614 13318 14181 15117 1551 16755 14078 3473
USER 10 RECOMMENDED!
3899 14358 6214 14400 4607 6975 13229 5273 16365 2243
USER 11 RECOMMENDED!
14181 3473 10865 11359 6431 249 12390 5577 1836 13134
USER 12 RECOMMENDED!
3899 6724 3200 1137 1792 15778 13963 14358 5925 13134
USER 13 RECOMMENDED!
8635 6214 15119 6975 

USER 110 RECOMMENDED!
6177 15181 17666 15119 17462 2152 13410 12943 13914 1259
USER 111 RECOMMENDED!
8580 5951 6976 11058 13585 6351 10857 9621 53 5714
USER 112 RECOMMENDED!
13134 13229 17416 16846 1792 17294 12186 3619 3473 9871
USER 113 RECOMMENDED!
5273 1551 8326 16364 10865 16012 6724 3200 4078 7224
USER 114 RECOMMENDED!
5403 13229 1137 7295 14358 16846 17294 17416 14781 10295
USER 115 RECOMMENDED!
7295 13914 3473 5424 13691 12707 739 7224 12378 287
USER 116 RECOMMENDED!
9621 17356 2665 8986 4559 5424 409 15800 14358 10095
USER 117 RECOMMENDED!
14358 15119 12752 5324 6214 8635 1619 16624 5299 3768
USER 118 RECOMMENDED!
17330 6782 3899 8635 249 1792 6975 15996 2665 5324
USER 119 RECOMMENDED!
3473 16365 17364 16946 13229 15750 512 9871 1259 13914
USER 120 RECOMMENDED!
8635 3942 2665 1259 3200 4088 11481 14868 14093 13318
USER 121 RECOMMENDED!
640 13963 12901 8635 2665 10865 6011 3433 1792 13229
USER 122 RECOMMENDED!
2665 12901 14868 17416 15778 1792 8207 640 4853 10865
USER 123 RECOM

USER 218 RECOMMENDED!
17294 13963 1137 5324 9871 13914 9131 7295 5299 13410
USER 219 RECOMMENDED!
8567 3889 10865 3200 12390 17356 12457 16012 14358 249
USER 220 RECOMMENDED!
13914 3899 1515 3889 1551 9871 8502 9441 10295 1653
USER 221 RECOMMENDED!
4582 9776 1259 14213 1137 1515 16846 16624 5299 2152
USER 222 RECOMMENDED!
4556 11359 249 757 2665 5186 640 17294 14213 3942
USER 223 RECOMMENDED!
6112 15119 8207 13549 13410 16170 1885 4914 4607 3200
USER 224 RECOMMENDED!
5273 7309 5403 5925 1941 5742 16486 16012 17135 13914
USER 225 RECOMMENDED!
8651 1259 1551 17330 14627 7407 13134 10295 1137 1614
USER 226 RECOMMENDED!
2665 16946 17330 11289 14266 3889 15868 12901 2055 6782
USER 227 RECOMMENDED!
14037 8635 17330 6214 11110 17135 16808 5273 12901 7870
USER 228 RECOMMENDED!
1941 14536 10562 1259 8326 17330 1208 4088 17882 13981
USER 229 RECOMMENDED!
6177 13410 13914 13725 4204 6011 15778 10316 4582 5403
USER 230 RECOMMENDED!
5324 17936 4582 12663 5221 8635 6177 15778 6214 1619
USER 231 RECO

USER 335 RECOMMENDED!
17364 7744 4582 6011 17330 985 2772 9131 15778 552
USER 336 RECOMMENDED!
13725 16070 6177 8635 12663 16791 11038 8037 17330 6589
USER 337 RECOMMENDED!
13410 3473 14358 14181 6975 4853 7407 12901 13644 8207
USER 338 RECOMMENDED!
15119 2886 9621 8011 14400 14181 3200 1614 16332 640
USER 339 RECOMMENDED!
1846 13295 17905 8580 5951 17291 16365 3899 3668 7606
USER 340 RECOMMENDED!
4088 2748 640 5605 14536 6975 3889 9610 15064 12172
USER 341 RECOMMENDED!
6905 7407 6214 5974 3889 8898 287 15750 6961 8326
USER 342 RECOMMENDED!
3473 2886 16846 10865 9131 1619 17330 8651 14873 5424
USER 343 RECOMMENDED!
16846 16624 8194 15082 16549 3768 6652 16131 11282 16221
USER 344 RECOMMENDED!
13229 12492 6177 5925 8936 17584 15398 5299 12311 13032
USER 345 RECOMMENDED!
16846 16365 4292 15119 4582 7264 5273 14396 6652 16624
USER 346 RECOMMENDED!
7716 3473 3899 11491 5925 419 6476 8567 2772 16012
USER 347 RECOMMENDED!
6214 12752 15778 17330 5509 10865 16846 5925 4292 16775
USER 348 RECOM

USER 459 RECOMMENDED!
13229 3433 15436 11359 13963 6975 5186 16624 2243 1619
USER 460 RECOMMENDED!
1846 6214 99 10865 17330 8194 15119 15289 14010 6905
USER 461 RECOMMENDED!
16365 8635 985 17330 16846 13410 7744 1792 1846 5273
USER 462 RECOMMENDED!
16365 13410 14358 4292 5925 3200 14181 12999 1614 16364
USER 463 RECOMMENDED!
2243 12707 16486 16364 5273 12943 6589 1605 13410 17022
USER 464 RECOMMENDED!
6177 14093 12707 7295 249 8326 14358 15119 5403 1515
USER 465 RECOMMENDED!
11504 5895 388 14396 7483 6177 7766 1186 13134 7190
USER 466 RECOMMENDED!
8609 9875 3910 9071 899 6247 16342 1015 13228 7074
USER 467 RECOMMENDED!
6214 3200 10865 17294 6975 12901 15119 5925 3768 1619
USER 468 RECOMMENDED!
13229 16364 1846 960 16846 5974 15750 7309 1496 419
USER 469 RECOMMENDED!
7716 8651 17553 14093 12311 16486 78 14996 7973 4853
USER 470 RECOMMENDED!
2886 5925 8326 6676 9275 16808 10680 3619 17135 7374
USER 471 RECOMMENDED!
8194 3473 5925 15551 15082 5433 11058 7143 1551 5981
USER 472 RECOMMENDED

...

USER 13088 RECOMMENDED!
10865 6975 9871 14358 13971 4204 15674 8037 16353 1614
USER 13089 RECOMMENDED!
1551 14181 17520 12311 17330 8567 3200 16364 16846 17943
USER 13090 RECOMMENDED!
17330 940 5424 249 14181 11038 5273 9373 1941 8651
USER 13091 RECOMMENDED!
16846 3899 4292 8635 17330 1551 10865 640 3889 5324
USER 13092 RECOMMENDED!
12901 6011 5640 11359 3822 10562 2748 10491 14814 5273
USER 13093 RECOMMENDED!
6214 14868 10865 13914 2665 16846 3768 17356 5403 5974
USER 13094 RECOMMENDED!
13725 8502 5925 241 6214 13465 3569 6905 1259 6782
USER 13095 RECOMMENDED!
16624 16846 6214 5324 985 13963 6975 16365 3473 6652
USER 13096 RECOMMENDED!
3473 15119 17388 13134 9871 13229 12186 14358 8207 241
USER 13097 RECOMMENDED!
9131 5885 4292 16486 2545 3899 1792 5273 13229 17670
USER 13098 RECOMMENDED!
12752 13229 6177 6975 1846 6157 9131 4292 4582 2665
USER 13099 RECOMMENDED!
4582 16846 12172 10295 15119 2746 5640 3473 16624 3899
USER 13100 RECOMMENDED!
5974 640 16791 4582 15778 14266 16365 5640 4

USER 13203 RECOMMENDED!
1725 4559 8871 15117 5273 4093 6905 7870 4853 13318
USER 13204 RECOMMENDED!
6905 14181 9744 17520 13584 16486 15469 8502 287 17330
USER 13205 RECOMMENDED!
16365 2152 14917 13229 3768 8635 5273 17330 14781 3889
USER 13206 RECOMMENDED!
9871 1614 2118 8567 4204 241 16353 7224 8651 1551
USER 13207 RECOMMENDED!
16070 1941 16364 14266 8747 6214 2772 7295 6011 13032
USER 13208 RECOMMENDED!
7374 16846 6177 17294 4078 1044 13229 14368 1137 7983
USER 13209 RECOMMENDED!
16365 3473 2665 17830 14358 8635 5324 4582 10865 1619
USER 13210 RECOMMENDED!
16549 10010 15082 14396 16365 15154 6214 3473 6975 17330
USER 13211 RECOMMENDED!
15119 12975 4559 11491 5273 8207 12186 17670 13131 8635
USER 13212 RECOMMENDED!
6214 17788 5885 5433 1614 1137 13229 17294 3955 4292
USER 13213 RECOMMENDED!
12752 10243 16365 11430 5273 5951 14181 1551 16846 14358
USER 13214 RECOMMENDED!
9441 1653 15969 6676 5925 241 17022 5742 17526 1259
USER 13215 RECOMMENDED!
6177 14358 15119 16365 4582 8635 5403 1

USER 13311 RECOMMENDED!
6214 14358 985 13229 16359 11844 16549 17330 2665 15082
USER 13312 RECOMMENDED!
5403 8326 2680 16365 1551 7412 12901 13887 17330 6431
USER 13313 RECOMMENDED!
10132 13134 14266 13691 3473 9619 512 2851 8096 2802
USER 13314 RECOMMENDED!
3619 7762 11481 8942 11339 6670 6214 3473 13549 17330
USER 13315 RECOMMENDED!
7295 6589 640 13725 15778 17330 16846 5424 1792 9131
USER 13316 RECOMMENDED!
4853 6975 5974 3473 3899 8207 14181 13134 17294 7744
USER 13317 RECOMMENDED!
17022 7295 5273 3889 12186 3768 6652 16846 1614 10370
USER 13318 RECOMMENDED!
16624 14037 1846 7483 12901 7606 17294 11227 6975 10865
USER 13319 RECOMMENDED!
3899 14868 5273 3200 10865 7975 1836 3829 1941 5403
USER 13320 RECOMMENDED!
3473 5925 1551 840 14093 6214 9275 12457 15469 15778
USER 13321 RECOMMENDED!
5324 13549 17364 12186 3899 13035 12943 985 15289 17666
USER 13322 RECOMMENDED!
13229 5273 13032 640 2665 1259 13691 2152 9131 1614
USER 13323 RECOMMENDED!
14181 3942 10295 1836 13318 7412 78 3200 1

USER 13442 RECOMMENDED!
5273 8635 15119 15778 6214 8011 5640 12390 13914 15996
USER 13443 RECOMMENDED!
3899 17330 17294 4204 16486 12457 1792 12752 2152 16549
USER 13444 RECOMMENDED!
17461 17584 4075 14396 17936 3899 5307 3473 6214 16751
USER 13445 RECOMMENDED!
17330 3899 1941 3889 6975 15796 15778 14358 5925 5086
USER 13446 RECOMMENDED!
17584 17330 10562 10370 17022 6214 13318 15778 13134 8326
USER 13447 RECOMMENDED!
17748 5925 13914 3473 1208 13134 12530 15469 4468 8567
USER 13448 RECOMMENDED!
17520 6724 13318 5273 11044 9913 3829 12127 4088 5320
USER 13449 RECOMMENDED!
1551 10865 3829 13410 1941 3200 9871 10562 8207 5424
USER 13450 RECOMMENDED!
13229 15119 14358 7744 4571 4220 16365 8635 13281 202
USER 13451 RECOMMENDED!
1614 8207 14358 13387 202 2665 12901 17330 13638 4582
USER 13452 RECOMMENDED!
13725 6975 2772 16353 11836 1551 13638 9441 15119 2886
USER 13453 RECOMMENDED!
8207 3899 13410 7716 12943 17364 13032 17330 8502 11175
USER 13454 RECOMMENDED!
6177 8635 6214 14358 4582 592

USER 13585 RECOMMENDED!
9131 2152 16791 6214 16359 16808 16486 3473 1259 16494
USER 13586 RECOMMENDED!
3473 1551 8635 17135 8651 5403 13914 14917 17748 17936
USER 13587 RECOMMENDED!
5273 5925 16364 17330 13691 13914 803 17526 17936 1551
USER 13588 RECOMMENDED!
8194 8207 15583 16846 5460 14181 7719 960 16359 4078
USER 13589 RECOMMENDED!
6177 16549 8011 7143 13229 7975 3768 13410 13134 9621
USER 13590 RECOMMENDED!
17584 3899 7257 10562 8326 8635 1551 14181 11576 5925
USER 13591 RECOMMENDED!
8635 1137 17330 11359 3829 1792 6011 4292 10370 16791
USER 13592 RECOMMENDED!
14396 16444 13295 14159 9898 8194 7867 15712 15289 6976
USER 13593 RECOMMENDED!
6011 5424 16012 3198 8473 2807 5714 4204 3889 6157
USER 13594 RECOMMENDED!
6011 8011 1619 2772 12901 15082 14226 53 14159 5433
USER 13595 RECOMMENDED!
6177 2243 6975 985 8207 13229 15778 249 5273 3899
USER 13596 RECOMMENDED!
6782 8502 3899 1137 1941 1551 12663 4088 16353 10865
USER 13597 RECOMMENDED!
3200 3899 8326 7716 7486 11339 4088 1259 17748